In [1]:
import numpy as np
import cv2

#얼굴, 눈 hearcascades, 9장
face_cascade = cv2.CascadeClassifier('./data/haarcascade_frontalface_default.xml')
mouth_cascade = cv2.CascadeClassifier('./data/haarcascade_mcs_mouth.xml')

#흑백 변환 경계값 조정 4.3장 스레시 홀딩
threshold = 80

#메세지
font = cv2.FONT_ITALIC
pos = (30, 30)
yes_mask_color = (255, 0, 0)
no_mask_color = (0, 0, 255)
yes_mask = "Thank you"
no_mask = "Please wear mask"

#캠 읽기
cam = cv2.VideoCapture(0)
while 1:
    #프레임 읽기
    ret, img = cam.read()
    img = cv2.flip(img, 1) #좌우반전
    
    #회색으로 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #흑백으로 변환
    (thresh, bw) = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    cv2.imshow('bw', bw)
    #얼굴 감지
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    faces_bw = face_cascade.detectMultiScale(bw, 1.1, 4)
    
    if(len(faces) == 0 and len(faces_bw) == 0): #얼굴이 감지가 안됨
        cv2.putText(img, "Not found face...", pos, font, 1, (255, 255, 255), 2)
    else: #얼굴이 감지됨
        for(x, y, w, h) in faces:
            cv2.rectangle(img,(x,y), (x + w, y + h), (255, 255, 255), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = img[y:y + h, x:x + w]
            #입감지
            mouth = mouth_cascade.detectMultiScale(gray, 1.5, 5)
            
        #입이 감지되지 않았을 때 마스크 씀
        if(len(mouth) == 0):
            cv2.putText(img, yes_mask, pos, font, 1, yes_mask_color, 2)
        else: #입이 감지됨
            #입술 검출로 입 확인
            for(mx, my, mw, mh) in mouth:
                if(y < my < y + h):
                    #입술이 입 위치에 있따면 마스크를 안쓴 것
                    cv2.putText(img, no_mask, pos, font, 1, no_mask_color, 2)
                    break
    
    #보여주기
    cv2.imshow('mask', img)
    a = cv2.waitKey(30)
    #esc누르면 종료
    if a == 27:
        break

#캠 종료
cam.release()
cv2.destoryAllWindows()

KeyboardInterrupt: 